# GraphRAG for Python

The [GraphRAG for Python](https://neo4j.com/docs/neo4j-graphrag-python/current/) package (neo4j-graphrag) allows you to access Neo4j Generative AI functions including:

- Retrievers

- GraphRAG pipelines

- Knowledge graph construction

The purpose is to provide a first party package to developers, where Neo4j can guarantee long term commitment and maintenance as well as being fast to ship new features and high performing patterns and methods.

You will use the neo4j-graphrag package to create retrievers and implement simple applications that use GraphRAG to provide context to LLM queries.

You must set up a development environment to run the code examples and exercises.


# Vector Retriever